In [ ]:
!pip install  -q 'ibis-framework[mssql,duckdb,deltalake]'
%pip install  -q altair

**_<u><mark>Get connection</mark></u>_**

In [2]:
import ibis
def get_ibis_connection(workspace ,lakehouse ,engine):
    import ibis
    import sempy.fabric as fabric
    workspaceID = fabric.resolve_workspace_id(workspace)
    df = fabric.list_items(workspace=workspace)
    df =df.loc[(df['Display Name'] == lakehouse) & (df['Type'] == 'Lakehouse')]
    lakehouseID = df['Id'].iloc[0]
    if engine =='dwh':
        import struct , pyodbc , ibis.backends.mssql
        dwh_backend = ibis.backends.mssql.Backend()
        client = fabric.FabricRestClient()
        response = client.get(f"/v1/workspaces/{workspaceID}/lakehouses/{lakehouseID}")
        responseJson = response.json()
        sql_endpoint = responseJson['properties']['sqlEndpointProperties']['connectionString']
        token = notebookutils.credentials.getToken('https://analysis.windows.net/powerbi/api').encode("UTF-16-LE")
        token_struct = struct.pack(f'<I{len(token)}s', len(token), token)
        dwh_backend.con = pyodbc.connect(f"Driver={{ODBC Driver 18 for SQL Server}};Server={sql_endpoint},1433;Encrypt=Yes;TrustServerCertificate=No", attrs_before={1256: token_struct})
        return dwh_backend
    else:
        access_token     = notebookutils.credentials.getToken('storage')
        storage_options  = {"bearer_token": access_token, "use_fabric_endpoint": "true"} 
        con = ibis.duckdb.connect(lakehouse)
        base_path = f'abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Tables'
        list_db = [item.name for item in  notebookutils.fs.ls(base_path)]
        for db in list_db:
           con.create_database(db,force =True)
           for table in [item.name for item in  notebookutils.fs.ls(base_path +f'/{db}')]:
             t= con.read_delta(base_path+f'/{db}/{table}',table,storage_options = storage_options)
             con.create_view(table, t, database=f'{lakehouse}.{db}')      
        return con

StatementMeta(, 3a3da5e1-3c98-41ca-9063-0c5ef870253e, 10, Finished, Available, Finished)

In [3]:
#switch between duckdb and dwh
con = get_ibis_connection(workspace ='pythononly',lakehouse ='NY',engine ='dwh')

StatementMeta(, 3a3da5e1-3c98-41ca-9063-0c5ef870253e, 11, Finished, Available, Finished)

In [4]:
con.list_catalogs()

StatementMeta(, 3a3da5e1-3c98-41ca-9063-0c5ef870253e, 12, Finished, Available, Finished)

['ETL', 'NY', 'TPCH', 'aemo', 'aemoraw', 'fabric_int_seasia', 'master']

In [5]:
con.list_databases(catalog = 'NY')

StatementMeta(, 3a3da5e1-3c98-41ca-9063-0c5ef870253e, 13, Finished, Available, Finished)

['INFORMATION_SCHEMA',
 '_rsc',
 'db_accessadmin',
 'db_backupoperator',
 'db_datareader',
 'db_datawriter',
 'db_ddladmin',
 'db_denydatareader',
 'db_denydatawriter',
 'db_owner',
 'db_securityadmin',
 'dbo',
 'guest',
 'ny',
 'queryinsights',
 'sys']

In [6]:
con.list_tables(database = 'NY.ny')

StatementMeta(, 3a3da5e1-3c98-41ca-9063-0c5ef870253e, 14, Finished, Available, Finished)

['taxi']

In [7]:
taxi = con.table(name="taxi", database='NY.ny')
agg = taxi.group_by("date").aggregate(TotalFares=taxi.fare_amount.sum(), AVGFares=taxi.fare_amount.mean())
ibis.to_sql(agg)

StatementMeta(, 3a3da5e1-3c98-41ca-9063-0c5ef870253e, 15, Finished, Available, Finished)

```sql
SELECT
  [t0].[date],
  SUM([t0].[fare_amount]) AS [TotalFares],
  AVG([t0].[fare_amount]) AS [AVGFares]
FROM [NY].[ny].[taxi] AS [t0]
GROUP BY
  [t0].[date]
```

In [8]:
data = agg.to_pyarrow()

StatementMeta(, 3a3da5e1-3c98-41ca-9063-0c5ef870253e, 16, Finished, Available, Finished)

In [9]:
import altair as alt
brush = alt.selection_interval()
details = alt.Chart(data).mark_bar().encode(alt.X('date:T'),  alt.Y('TotalFares:Q'), tooltip=[alt.Tooltip('date:T',format='%Y-%m-%d %H'),'TotalFares:Q']
).properties( width=1400,  height=400 ).add_params( brush)

summary = alt.Chart(data).mark_square().encode( alt.X('date:T'), alt.Y('AVGFares:Q'), tooltip=['AVGFares:Q'] ).properties(  width=1400,  height=400).transform_filter( brush)
details & summary

StatementMeta(, 3a3da5e1-3c98-41ca-9063-0c5ef870253e, 17, Finished, Available, Finished)

alt.VConcatChart(...)